In [1]:
def gf(eps, m_V, Pms, chir):
    e = mt.sqrt(4*mt.pi/137.036)
    m_Z = 91.2 #GeV
    Sw = mt.sqrt(0.22305)
    Cw = mt.sqrt(1-0.22305)
    if chir == 'R':
        t3 = 0
        Yf = Pms[2][0]/2
    elif chir == 'L' and Pms[-1]:
        t3 = 1/2
        Yf = Pms[2][1]/2
    else:
        t3 = -1/2
        Yf = Pms[2][1]/2
    eta = eps/(Cw*np.sqrt(1-(eps/Cw)**2))
    delta = m_V/m_Z
    g = np.zeros(len(m_V))
    alpha = np.zeros(len(m_V))
    alpha[delta == 1] = -eta*Sw/2
    alpha = np.delete(alpha, delta == 1)
    deltap = delta[delta != 1]
    alpha[delta != 1] = np.arctan((1 - eta**2*Sw**2 - deltap**2 - np.sign(1-deltap**2)
                                   *np.sqrt(4*eta**2*Sw**2 + (1 - eta**2*Sw**2 - deltap**2)**2))/(2*eta*Sw))
#     g[delta == 1] = eps*e*(t3*Cw**2 - Yf*Sw**2)/(Cw*Sw)
#     g[delta != 1] = e*(-np.sin(alpha)*(t3*Cw**2 - Yf*Sw**2) + eta*np.cos(alpha)*Sw*Yf)/(Cw*Sw)
    g = e*(-np.sin(alpha)*(t3*Cw**2 - Yf*Sw**2) + eta*np.cos(alpha)*Sw*Yf)/(Cw*Sw)
    return g

def GammaLO(m_V, eps, Pms): # Pms = [ m_f, Qf, Yf, Nf, True/False ] element 2 in Pms must be a length 2 vector
    if not isinstance(m_V, Iterable): m_V = np.array([m_V])
    if len(m_V) > 1: 
        Gamma = np.zeros(len(m_V))
        m_Vp = m_V[m_V > 2*Pms[0]]
        gfR = gf(eps,m_Vp,Pms,'R')
        gfL = gf(eps,m_Vp,Pms,'L')
        Gamma[m_V > 2*Pms[0]] = Pms[3]/(24*mt.pi)*m_Vp*np.sqrt(1 - 4*(Pms[0]/m_Vp)**2)*(gfR**2 + gfL**2 - (Pms[0]/m_Vp)**2*(gfR**2 + gfL**2 - 6*gfR*gfL))
    elif m_V[0] > 2*Pms[0]:
        gfR = gf(eps,m_V,Pms,'R')
        gfL = gf(eps,m_V,Pms,'L')
        Gamma = Pms[3]/(24*mt.pi)*m_V*np.sqrt(1 - 4*(Pms[0]/m_V)**2)*(gfR**2 + gfL**2 - (Pms[0]/m_V)**2*(gfR**2 + gfL**2 - 6*gfR*gfL))
    else:
        #print(r'Mass of the mediating particle, $m_{V}$, must be greater or equal to $2\cdot m_f$')
        Gamma = np.array([0.0])
#     if Pms[0] == m_nu:
#         m_Vp = m_V[m_V <= 2*m_nu]
#         mf = 1e-9
#         Gamma[m_V <= 2*m_nu] = Pms[3]/(24*mt.pi)*m_Vp*np.sqrt(1 - 4*(mf/m_Vp)**2)*(gf(eps, m_Vp, Pms, 'R')**2 + gf(eps, m_Vp, Pms, 'L')**2 - (mf/m_Vp)**2*(gf(eps, m_Vp, Pms, 'R')**2 + gf(eps, m_Vp, Pms, 'L')**2 - 6*gf(eps, m_Vp, Pms, 'R')*gf(eps, m_Vp, Pms, 'L')))
#     else:
#         Gamma[m_V <= 2*Pms[0]] = np.zeros(len(m_V[m_V <= 2*Pms[0]]))
    ConvToSI = hbar*c
    return Gamma/ConvToSI#*1.02928886#*1.0931143929657048 # Gamma in m^-1

def GammaTot(m_V, eps, PMS):
    if not isinstance(m_V, Iterable): m_V = np.array([m_V])
    GammaTotal = np.zeros(len(m_V))
    for i in range(6):
        GammaTotal = GammaLO(m_V, eps, PMS[i]) + GammaTotal
    GammaVq = np.zeros(len(m_V))
    for i in range(3):
        GammaVq = GammaLO(m_V, eps, PMS[6 + i]) + GammaVq
    GammaVq[m_V < 5] = np.zeros(len(m_V[m_V < 5]))
    GammaTotal = GammaVq + GammaTotal
    for i in range(9,len(PMS)):
        GammaTotal = GammaLO(m_V, eps, PMS[i]) + GammaTotal
    return GammaTotal